In [1]:
import sys
import os
sys.path.append('/g/data/jk72/ll6859/ACCESS_aero_eval/')
from aercode import *
import matplotlib 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import xarray as xr
import glob as gb
import cartopy.crs as ccrs
import seaborn as sns
import scipy.special
from datetime import datetime

import dask
import warnings
from dask.distributed import Client

In [2]:
plt.style.use('default')
warnings.filterwarnings('ignore')

# RVInvestigator

bx400

In [4]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/RVInvestigator/'
aer = pd.read_csv(f_path+'rvigaw_cn10_2016to2019_L2.csv', index_col='datetime')
aer['time'] = pd.DatetimeIndex(aer.index)
aer.index = aer['time']
aer = aer.resample('1D', kind='time').mean()
aer = aer[['cn10_avg','cn10_std','latitude','longitude']].dropna()
aer.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)


met = pd.read_csv(f_path+'RVI-met.csv', delimiter=',', skiprows=54, index_col='TIME')
met['time'] = pd.DatetimeIndex(met.index)
met.index = met['time'].astype('datetime64[ns]')
met = met[['AIRT_1','ATMP','RAIN_AMOUNT_1','WDIR_1','WSPD_1']]
met['RAIN_AMOUNT_1'] = met['RAIN_AMOUNT_1'].diff()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['RAIN_AMOUNT_1'] = met_tmp['RAIN_AMOUNT_1'].resample('1D', kind='time').sum()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']>60]=np.nan
met.rename(columns={'AIRT_1': 'air_temp', 'RAIN_AMOUNT_1': 'total_rain', 'ATMP': 'air_pressure', 'WDIR_1': 'wind_dir', 'WSPD_1': 'wind_spd'}, inplace=True)

rdn = pd.read_excel(f_path+'2020_05_29d_RVI_1420_Radon.xlsx', sheet_name='RVI1420', index_col='DateTime')
rdn['time'] = pd.DatetimeIndex(rdn.index)
rdn.index = rdn['time']
rdn = rdn[rdn['Radon']>0]
rdn = rdn.resample('1D', kind='time').mean()
rdn = rdn[['Radon','[SD]','RnSTP']].dropna()
rdn.rename(columns={'Radon': 'radon', '[SD]': 'radon_std', 'RnSTP': 'radon_stp'}, inplace=True)

In [5]:
# convert observations to a dataset
obs = aer.to_xarray()
obs = obs.assign(met)
obs = obs.assign(rdn)

In [7]:
# load in model data
rvi_bx400 = xr.open_dataset(f_path+'rvi1619_bx400_track.nc')

In [8]:
# merge model and obs
rvi_df = xr.merge([rvi_bx400,obs],compat='override')

# remove outliers
rvi_df = rvi_df.where(rvi_df['cn10_avg']<10000)
rvi_df = rvi_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
rvi_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations along the 2016-2019 RV Investigator voyage tracks'
rvi_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [10]:
rvi_df.load().to_netcdf(path=f_path+'rvi1619_bx400_df.nc')

cg283

In [11]:
# load in model data
rvi_cg283 = xr.open_dataset(f_path+'rvi1619_cg283_track.nc')

In [12]:
# merge model and obs
rvi_df = xr.merge([rvi_cg283,obs],compat='override')

# remove outliers
rvi_df = rvi_df.where(rvi_df['cn10_avg']<10000)
rvi_df = rvi_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
rvi_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2016-2019 RV Investigator voyage tracks'
rvi_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [13]:
rvi_df.load().to_netcdf(path=f_path+'rvi1619_cg283_df.nc')

# Cold Water Trial

bx400

In [14]:
# f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/ColdWaterTrial/'

# df = pd.read_csv(f_path+'CWT_ProcessedAerosolData_1min.csv', delimiter=',', index_col='date')
# df['time'] = pd.DatetimeIndex(df.index)
# df.index = df['time'].astype('datetime64[ns]')
# df = df[['lat','lon','AirTemp_port','AirTemp_stbd','atmPress','WindSpdTru','Radon_hrly','CN_mean']]
# df = df.dropna()
# df = df.resample('1D', kind='time').mean()

# Reef 2 Rainforest

bx400

In [15]:
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/Reef2Rain/'

df = pd.read_csv(f_path+'R2R_RV_aerosol.csv', index_col='date')
df['time'] = pd.DatetimeIndex(df.index)
df.index = df['time'].astype('datetime64[ns]')
df = df.resample('1D', kind='time').mean()
df = df.dropna()
df['air_temp'] = (df['AirTemp_port']+df['AirTemp_stbd'])/2
df['wind_spd'] = (df['WindSpdTru_port']+df['WindSpdTru_stbd'])/2
df = df[['lat','lon','air_temp','atmPress','WindDir', 'wind_spd', 'Radon_hrly','CN3_mean','CN10_mean','CCN_mean']]
df.rename(columns={'atmPress': 'air_pressure', 
                   'WindDir': 'wind_dir', 
                   'Radon_hrly': 'radon', 
                   'CN3_mean': 'cn3_avg', 
                   'CN10_mean': 'cn10_avg', 
                   'CCN_mean': 'ccn_avg'}, inplace=True)

In [16]:
# convert observations to a dataset
obs = df.to_xarray()

In [17]:
# load in model data
r2r_bx400 = xr.open_dataset(f_path+'r2r16_bx400_track.nc')

In [18]:
# merge model and obs
r2r_df = xr.merge([r2r_bx400,obs],compat='override')

# remove outliers
r2r_df = r2r_df.where(r2r_df['cn10_avg']<10000)
r2r_df = r2r_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
r2r_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2016 Reef 2 Rainforest voyage tracks'
r2r_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [19]:
r2r_df.load().to_netcdf(path=f_path+'r2r16_bx400_df.nc')

cg283

In [20]:
# load in model data
r2r_cg283 = xr.open_dataset(f_path+'r2r16_cg283_track.nc')

In [21]:
# merge model and obs
r2r_df = xr.merge([r2r_cg283,obs],compat='override')

# remove outliers
r2r_df = r2r_df.where(r2r_df['cn10_avg']<10000)
r2r_df = r2r_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
r2r_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2016 Reef 2 Rainforest voyage tracks'
r2r_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [22]:
r2r_df.load().to_netcdf(path=f_path+'r2r16_cg283_df.nc')

# Ice 2 Equator

bx400

In [89]:
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/Ice2Equator/'

df = pd.read_csv(f_path+'I2E_ProcessedAerosolData_1min.csv', index_col='date')
df['time'] = pd.DatetimeIndex(df.index)
df.index = df['time'].astype('datetime64[ns]')
df = df.resample('1D', kind='time').mean()
df = df.dropna()
df['air_temp'] = (df['AirTemp_port']+df['AirTemp_stbd'])/2
df['wind_spd'] = (df['WindSpdTru_port']+df['WindSpdTru_stbd'])/2
df = df[['lat','lon','air_temp','atmPress','wind_spd','Radon_hrly','CN_mean','CN_std']]
df.rename(columns={'atmPress': 'air_pressure',
                   'Radon_hrly': 'radon', 
                   'CN_mean': 'cn3_avg', 
                   'CN_std': 'cn3_std'}, inplace=True)

In [91]:
# convert observations to a dataset
obs = df.to_xarray()

In [92]:
# load in model data
i2e_bx400 = xr.open_dataset(f_path+'i2e16_bx400_track.nc')

In [96]:
# merge model and obs
i2e_df = xr.merge([i2e_bx400,obs],compat='override')

# remove outliers
i2e_df = i2e_df.where(i2e_df['cn3_avg']<10000)
i2e_df = i2e_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
i2e_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations along the 2016 Ice 2 Equator voyage tracks'
i2e_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [98]:
i2e_df.load().to_netcdf(path=f_path+'i2e16_bx400_df.nc')

cg283

In [99]:
# load in model data
i2e_cg283 = xr.open_dataset(f_path+'i2e16_cg283_track.nc')

In [100]:
# merge model and obs
i2e_df = xr.merge([i2e_cg283,obs],compat='override')

# remove outliers
i2e_df = i2e_df.where(i2e_df['cn3_avg']<10000)
i2e_df = i2e_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
i2e_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2016 Ice 2 Equator voyage tracks'
i2e_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [101]:
i2e_df.load().to_netcdf(path=f_path+'i2e16_cg283_df.nc')

# CAPRICORN I

bx400

In [130]:
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/Capricorn/'

aer = pd.read_hdf(f_path+'aerosol_1sec.h5')
aer.index.name = 'time'
aer = aer.resample('1D', kind='time').mean()
aer.rename(columns={'ccn_0.5504': 'ccn_avg',
                   'ccn_sigma': 'ccn_std', 
                   'cn3': 'cn3_avg', 
                   'cn3_sigma': 'cn3_std'}, inplace=True)

met = pd.read_csv(f_path+'in2016_v02uwy10sec.csv', parse_dates=[['date', 'time']], index_col='date_time')
met = met.resample('1D').mean()
met['air_temp'] = (met['portAirTemp(degC)']+met['stbdAirTemp(degC)'])/2
met = met[['latitude(degree_north)','longitude(degree_east)','air_temp','atmPressure(mbar)','rain(mm)']]
met.rename(columns = {"latitude(degree_north)": "lat",
                      "longitude(degree_east)": "lon",
                      "atmPressure(mbar)": "air_pressure",
                      "rain(mm)": "total_rain"}, inplace=True)
met.index.name = 'time'   

In [131]:
# convert observations to a dataset
obs = aer.to_xarray()
obs = obs.assign(met)

In [133]:
# load in model data
cap1_bx400 = xr.open_dataset(f_path+'cap16_bx400_track.nc')

In [135]:
# merge model and obs
cap1_df = xr.merge([cap1_bx400,obs],compat='override')

# remove outliers
cap1_df = cap1_df.where(cap1_df['cn3_avg']<10000)
cap1_df = cap1_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
cap1_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations along the 2016 CAPRICORN I voyage tracks'
cap1_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [137]:
cap1_df.load().to_netcdf(path=f_path+'cap16_bx400_df.nc')

cg283

In [138]:
# load in model data
cap1_cg283 = xr.open_dataset(f_path+'cap16_cg283_track.nc')

In [139]:
# merge model and obs
cap1_df = xr.merge([cap1_cg283,obs],compat='override')

# remove outliers
cap1_df = cap1_df.where(cap1_df['cn3_avg']<10000)
cap1_df = cap1_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
cap1_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2016 CAPRICORN I voyage tracks'
cap1_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [140]:
cap1_df.load().to_netcdf(path=f_path+'cap16_cg283_df.nc')

# CAPRICORN II

bx400

In [206]:
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/Capricorn/'

aer = pd.read_csv(f_path+'capricorn2_cpc3772_L2_1H.csv', index_col=0)
aer.index.name = 'time'
aer['time'] = pd.DatetimeIndex(aer.index)
aer.index = aer['time'].astype('datetime64[ns]')
aer = aer.resample('1D', kind='time').mean()
aer = aer[['cn_avg', 'cn_std']]
aer.rename(columns={'cn_avg': 'cn10_avg',
                   'cn_std': 'cn10_std'}, inplace=True)

met = pd.read_csv(f_path+'in2018_v01uwy10sec.csv', parse_dates=[['date', 'time']], index_col='date_time')
for v in met.select_dtypes('object'):
    met[v] = pd.to_numeric(met[v], errors='coerce')
met.index.name = 'time'
met = met.resample('1D').mean()
met['air_temp'] = (met['portAirTemp(degC)']+met['stbdAirTemp(degC)'])/2
met['total_rain'] = (met['portRain(mm)']+met['stbdRain(mm)'])/2
met['wind_spd'] = (met['portTrueWindSpeed(knot)']+met['stbdTrueWindSpeed(knot)'])/2*1.944
met['wind_dir'] = (met['portTrueWindDir(degree)']+met['stbdTrueWindDir(degree)'])/2
met = met[['latitude(degree_north)','longitude(degree_east)','air_temp','atmPressure(mbar)','total_rain','wind_spd','wind_dir']]
met.rename(columns = {"latitude(degree_north)": "lat",
                            "longitude(degree_east)": "lon",
                            "atmPressure(mbar)": "air_pressure"}, inplace=True)

In [207]:
# convert observations to a dataset
obs = aer.to_xarray()
obs = obs.assign(met)

In [208]:
# load in model data
cap2_bx400 = xr.open_dataset(f_path+'cap18_bx400_track.nc')

In [209]:
# merge model and obs
cap2_df = xr.merge([cap2_bx400,obs],compat='override')

# remove outliers
cap2_df = cap2_df.where(cap2_df['cn10_avg']<10000)
cap2_df = cap2_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
cap2_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations along the 2018 CAPRICORN II voyage tracks'
cap2_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [211]:
cap2_df.load().to_netcdf(path=f_path+'cap18_bx400_df.nc')

cg283

In [212]:
# load in model data
cap2_cg283 = xr.open_dataset(f_path+'cap18_cg283_track.nc')

In [213]:
# merge model and obs
cap2_df = xr.merge([cap2_cg283,obs],compat='override')

# remove outliers
cap2_df = cap2_df.where(cap2_df['cn10_avg']<10000)
cap2_df = cap2_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
cap2_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2018 CAPRICORN II voyage tracks'
cap2_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [214]:
cap2_df.load().to_netcdf(path=f_path+'cap18_cg283_df.nc')

# MARCUS

bx400

In [34]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/AuroraAustralis/'

aer = pd.read_csv(f_path+'CN_MARCUS_1hr.csv', index_col=0)
aer.index.name = 'time'
aer['time'] = pd.DatetimeIndex(aer.index)
aer.index = aer['time']
aer = aer.resample('1D', kind='time').mean()
aer = aer[['cn_avg','cn_std','lat','lon']].dropna()
aer.rename(columns={'cn_avg': 'cn10_avg', 'cn_std': 'cn10_std'}, inplace=True)

met = pd.read_csv(f_path+'marcus-met.csv', delimiter=',', skiprows=54, index_col='TIME')
met['time'] = pd.DatetimeIndex(met.index)
met.index = met['time'].astype('datetime64[ns]')
met = met[['AIRT_1','ATMP','RAIN_AMOUNT_1','WDIR_1','WSPD_1']]
met['RAIN_AMOUNT_1'] = met['RAIN_AMOUNT_1'].diff()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['RAIN_AMOUNT_1'] = met_tmp['RAIN_AMOUNT_1'].resample('1D', kind='time').sum()
met.rename(columns={'AIRT_1': 'air_temp', 'RAIN_AMOUNT_1': 'total_rain', 'ATMP': 'air_pressure', 'WDIR_1': 'wind_dir', 'WSPD_1': 'wind_spd'}, inplace=True)

In [35]:
# convert observations to a dataset
obs = aer.to_xarray()
obs = obs.assign(met)

In [37]:
# load in model data
aa_bx400 = xr.open_dataset(f_path+'aa1718_bx400_track.nc')

In [38]:
# merge model and obs
aa_df = xr.merge([aa_bx400,obs],compat='override')

# remove outliers
aa_df = aa_df.where(aa_df['cn10_avg']<10000)
aa_df = aa_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
aa_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations along the 2017-2018 MARCUS voyage tracks'
aa_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [39]:
aa_df.load().to_netcdf(path=f_path+'aa1718_bx400_df.nc')

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


cg283

In [41]:
# load in model data
aa_cg283 = xr.open_dataset(f_path+'aa1718_cg283_track.nc')

In [43]:
# merge model and obs
aa_df = xr.merge([aa_cg283,obs],compat='override')

# remove outliers
aa_df = aa_df.where(aa_df['cn10_avg']<10000)
aa_df = aa_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
aa_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2017-2018 MARCUS voyage tracks'
aa_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [44]:
aa_df.load().to_netcdf(path=f_path+'aa1718_cg283_df.nc')

# TAN1802

bx400

In [52]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/TAN1802/'
aer = xr.open_dataset(f_path+'CPC3010_TAN1802_2018_level_01.nc')
aer = aer.rename({'number_concentration_of_ambient_aerosol_particles_in_air':'cn10',
                  'number_concentration_of_ambient_aerosol_particles_in_air_flag':'cn10_flag'})
aer = aer.where(aer.cn10_flag == 1)
aer['cn10'] = aer['cn10']/1000000
aer = aer.resample(time='1D').mean()
aer = aer.rename({'cn10': 'cn10_avg'})

uw = keycutter(f_path+'weather.csv','Time (UTC)', 'Latitude', 'Longitude')


met = pd.read_csv(f_path+'TAN-met.csv', delimiter=',', skiprows=54, index_col='TIME')
met['time'] = pd.DatetimeIndex(met.index)
met.index = met['time'].astype('datetime64[ns]')
met = met[['AIRT_1','ATMP','RAIN_AMOUNT_1','WDIR_1','WSPD_1']]
met['RAIN_AMOUNT_1'] = met['RAIN_AMOUNT_1'].diff()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['RAIN_AMOUNT_1'] = met_tmp['RAIN_AMOUNT_1'].resample('1D', kind='time').sum()
met.rename(columns={'AIRT_1': 'air_temp', 'RAIN_AMOUNT_1': 'total_rain', 'ATMP': 'air_pressure', 'WDIR_1': 'wind_dir', 'WSPD_1': 'wind_spd'}, inplace=True)

In [53]:
# convert observations to a dataset
obs = aer
obs = obs.assign(uw)
obs = obs.assign(met)

In [54]:
# load in model data
tan_bx400 = xr.open_dataset(f_path+'tan1718_bx400_track.nc')

In [55]:
# merge model and obs
tan_df = xr.merge([tan_bx400,obs],compat='override')

# remove outliers
tan_df = tan_df.where(tan_df['cn10_avg']<10000)
tan_df = tan_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
tan_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations along the 2017-2018 TAN1802 voyage tracks'
tan_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [56]:
tan_df.load().to_netcdf(path=f_path+'tan1718_bx400_df.nc')

cg283

In [57]:
# load in model data
tan_cg283 = xr.open_dataset(f_path+'tan1718_cg283_track.nc')

In [58]:
# merge model and obs
tan_df = xr.merge([tan_cg283,obs],compat='override')

# remove outliers
tan_df = tan_df.where(tan_df['cn10_avg']<10000)
tan_df = tan_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
tan_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations along the 2017-2018 TAN1802 voyage tracks'
tan_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [60]:
tan_df.load().to_netcdf(path=f_path+'tan1718_cg283_df.nc')

# Cape Grim

bx400

In [62]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/CGO/'
aer = pd.read_excel(f_path+'cgbpaps_CN  2016-2018.xlsx', index_col='date')
aer = aer.astype(np.float64).resample('1D',kind='time').mean()
aer.index.name = 'time'
aer = aer[['cn3772_mean','cn3772_sd']]
aer.rename(columns={'cn3772_mean': 'cn10_avg', 'cn3772_sd': 'cn10_std'}, inplace=True)

met = pd.read_excel(f_path+'CG_met_2015-2018.xlsx', index_col='date')
met = met[['TempGC','Pressure','Rain','VWS10m','VWD10m']]
met.index.name = 'time'
met = met.resample('1D', kind='time').mean()
met.rename(columns={'TempGC': 'air_temp', 'Rain': 'total_rain', 'Pressure': 'air_pressure', 'VWD10m': 'wind_dir', 'VWS10m': 'wind_spd'}, inplace=True)

rdn = pd.read_excel(f_path+'2021_08_25a_CGO_Radon_2015-2018_ForLiam.xlsx', sheet_name='2015-2018-RadonMetCO2', index_col='DateTime')
rdn.index.name = 'time'
rdn = rdn[rdn['Radon']>0]
rdn = rdn.resample('1D', kind='time').mean()
rdn = rdn[['Radon','[SD]']].dropna()
rdn.rename(columns={'Radon': 'radon', '[SD]': 'radon_std'}, inplace=True)

In [63]:
# convert observations to a dataset
obs = aer.to_xarray()
obs = obs.assign(met)
obs = obs.assign(rdn)

In [66]:
# load in model data
cgo_bx400 = xr.open_dataset(f_path+'cgo1618_bx400_track.nc')

In [67]:
# merge model and obs
cgo_df = xr.merge([cgo_bx400,obs],compat='override')

# remove outliers
cgo_df = cgo_df.where(cgo_df['cn10_avg']<10000)
cgo_df = cgo_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
cgo_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations at the 2016-2018 Cape Grim Baseline Air Pollution Station'
cgo_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [69]:
cgo_df.load().to_netcdf(path=f_path+'cgo1618_bx400_df.nc')

cg283

In [70]:
# load in model data
cgo_cg283 = xr.open_dataset(f_path+'cgo1618_cg283_track.nc')

In [71]:
# merge model and obs
cgo_df = xr.merge([cgo_cg283,obs],compat='override')

# remove outliers
cgo_df = cgo_df.where(cgo_df['cn10_avg']<10000)
cgo_df = cgo_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
cgo_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations at the 2016-2018 Cape Grim Baseline Air Pollution Station'
cgo_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [73]:
cgo_df.load().to_netcdf(path=f_path+'cgo1618_cg283_df.nc')

# Macquarie Island

bx400

In [74]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/MQI/'
aer = pd.read_hdf(f_path+'ACRE_CN10_1H_FINAL.h5')
aer.index.name = 'time'
aer = aer[['cn_avg','cn_std']]
aer = aer.resample('1D', kind='time').mean()
aer.rename(columns={'cn_avg': 'cn10_avg', 'cn_std': 'cn10_std'}, inplace=True)

met = []
all_files = gb.glob(f_path+'AWS_hourly/*.csv')
met = pd.concat([pd.read_csv(v) for v in all_files])
met.index = met['MIN_OBSERVATION_DATE_'].astype('datetime64[ns]')
met.index.name = 'time'
met['AVG_RAINFALL'] = met['AVG_RAINFALL'].diff()
met['AVG_RAINFALL'][met['AVG_RAINFALL']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['AVG_RAINFALL'] = met_tmp['AVG_RAINFALL'].resample('1D', kind='time').sum()
met = met[['AVG_AIR_TEMPERATURE','AVG_WIND_DIRECTION','AVG_WIND_SPEED','AVG_AIR_PRESSURE','AVG_RAINFALL']]
met.rename(columns={'AVG_AIR_TEMPERATURE': 'air_temp', 'AVG_RAINFALL': 'total_rain', 'AVG_AIR_PRESSURE': 'air_pressure', 'AVG_WIND_DIRECTION': 'wind_dir', 'AVG_WIND_SPEED': 'wind_spd'}, inplace=True)

rdn = pd.read_excel(f_path+'2021_09_15a_MQI_Radon_2013-2019_ForLiamLamprey.xlsx',
                    sheet_name='Radon_MQI_2013-2019',
                    index_col='DateTime')
rdn.index.name = 'time'
rdn = rdn[rdn['Radon']>0]
rdn = rdn.resample('1D', kind='time').mean()
rdn = rdn[['Radon','[SD]']].dropna()
rdn.rename(columns={'Radon': 'radon', '[SD]': 'radon_std'}, inplace=True)

In [75]:
# convert observations to a dataset
obs = aer.to_xarray()
obs = obs.assign(met)
obs = obs.assign(rdn)

In [79]:
# load in model data
mqi_bx400 = xr.open_dataset(f_path+'mqi1618_bx400_track.nc')

In [80]:
# merge model and obs
mqi_df = xr.merge([mqi_bx400,obs],compat='override')

# remove outliers
mqi_df = mqi_df.where(mqi_df['cn10_avg']<10000)
mqi_df = mqi_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
mqi_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations at the 2016-2018 Macquarie Island Research Station'
mqi_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [81]:
mqi_df.load().to_netcdf(path=f_path+'mqi1618_bx400_df.nc')

cg283

In [82]:
# load in model data
mqi_cg283 = xr.open_dataset(f_path+'mqi1618_cg283_track.nc')

In [85]:
# merge model and obs
mqi_df = xr.merge([mqi_cg283,obs],compat='override')

# remove outliers
mqi_df = mqi_df.where(mqi_df['cn10_avg']<10000)
mqi_df = mqi_df.drop('z1_hybrid_height').squeeze()

# Edit data attributes
mqi_df.attrs['title'] = 'ACCESS-AM2 model output (cg283) and observations at the 2016-2018 Macquarie Island Research Station'
mqi_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [86]:
mqi_df.load().to_netcdf(path=f_path+'mqi1618_cg283_df.nc')